<a href="https://colab.research.google.com/github/unknown-jun/Basic_of_Python/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

서포트 벡터 머신은 기계학습 분야의 하나로 패턴 인식, 자료분석을 위한 지도학습 모델이며 주로 분류와 회귀분석을 위해 사용함.  
서포트 벡터 머신 알고리즘은 데이터를 분류하는 가장 큰 폭을 가진 경계를 찾는 알고리즘.  
비선형 분류를 하기 위해서 주어진 데이터를 고차원 특징 공간으로 사상하는 작업이 필요한데, 이를 효율적으로 하기 위해 커널 트릭을 사용하기도 한다.

# 타이타닉을 이용한 SVM

In [ ]:
### 기본 라이브러리 불러오기
import pandas as pd
import seaborn as sns


In [ ]:
'''
[Step 1] 데이터 준비/ 기본 설정
'''
# load_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')

#  IPython 디스플레이 설정 - 출력할 열의 개수 한도 늘리기
pd.set_option('display.max_columns', 15)


In [ ]:
'''
[Step 2] 데이터 탐색/ 전처리
'''

# NaN값이 많은 deck 열을 삭제, embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)  

# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)  

# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()   
rdf['embarked'].fillna(most_freq, inplace=True)


In [ ]:
'''
[Step 3] 분석에 사용할 속성을 선택
'''

# 분석에 활용할 열(속성)을 선택 
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]
ndf

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S
...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,Q
886,0,2,male,27.0,0,0,S
887,1,1,female,19.0,0,0,S
889,1,1,male,26.0,0,0,C


In [ ]:

# 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex', 'embarked'], axis=1, inplace=True)


In [ ]:
'''
[Step 4] 데이터셋 구분 - 훈련용(train data)/ 검증용(test data)
'''

# 속성(변수) 선택
X=ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 
       'town_C', 'town_Q', 'town_S']]  #독립 변수 X
y=ndf['survived']                      #종속 변수 Y

In [ ]:
# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)
print('\n')

train data 개수:  (499, 9)
test data 개수:  (215, 9)




In [ ]:
'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''

# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')
svm_model

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
y_hat = svm_model.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 1 0 0 0 1 0 0 0]
[0 0 1 0 0 1 1 1 0 0]


In [ ]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, y_hat)  
print(svm_matrix)
print('\n')


# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, y_hat)            
print(svm_report)

[[120   5]
 [ 35  55]]


              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215



문제. 지금까지의 정확도는 0.81인데 아이와 여자 먼저라는 파생변수를 추가하면 정확도가 더 올라가는지 확인하시오

In [ ]:
# load_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')

# NaN값이 많은 deck 열을 삭제, embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)  

# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)  

# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()   
rdf['embarked'].fillna(most_freq, inplace=True)

# 파생변수 생성
mask = ( rdf.age < 10 ) | ( rdf.sex=='female')
mask.astype(int) # True 를 1 로 변경하고 False 를 0 으로 변경
rdf['women_child'] = mask.astype(int)

# 가정2: fare가 0인 인원은 선원일 것이며 대다수의 남자 선원은 죽었을 것이다.
mask = ( rdf.fare == 0 ) & ( rdf.sex== 'male' )
rdf['male_fare0'] = mask.astype(int)

# 가정3: 3등실 남자 인원은 모두 죽었을 것이다.
mask = ( rdf.pclass != 3 ) & ( rdf.sex== 'male' )
rdf['male_Third'] = mask.astype(int)

# 가정4: 50살 이상의 남자는 대다수가 죽었을 것이다.
mask = ( rdf.age < 50 ) & ( rdf.sex== 'male' )
rdf['male_age50'] = mask.astype(int)

# 분석에 활용할 열(속성)을 선택 
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked','women_child','male_fare0','male_Third','male_age50']]


# 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex', 'embarked'], axis=1, inplace=True)


# 속성(변수) 선택
X=ndf[['pclass', 'age', 'sibsp', 'parch',
       'women_child','male_fare0','male_Third','male_age50']]  #독립 변수 X
y=ndf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
y_hat = svm_model.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, y_hat)  
print(svm_matrix)
print('\n')

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, y_hat)            
print(svm_report)

[[119   6]
 [ 29  61]]


              precision    recall  f1-score   support

           0       0.80      0.95      0.87       125
           1       0.91      0.68      0.78        90

    accuracy                           0.84       215
   macro avg       0.86      0.81      0.82       215
weighted avg       0.85      0.84      0.83       215



문제. grid search 를 이용해서 최적의 하이퍼 파라미터의 조합을 알아내시오

In [ ]:
# load_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')

# NaN값이 많은 deck 열을 삭제, embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)  

# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)  

# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()   
rdf['embarked'].fillna(most_freq, inplace=True)

# 파생변수 생성
mask = ( rdf.age < 10 ) | ( rdf.sex=='female')
mask.astype(int) # True 를 1 로 변경하고 False 를 0 으로 변경
rdf['women_child'] = mask.astype(int)

# 가정2: fare가 0인 인원은 선원일 것이며 대다수의 남자 선원은 죽었을 것이다.
mask = ( rdf.fare == 0 ) 
rdf['male_fare0'] = mask.astype(int)

# 가정3: 3등실 남자 인원은 모두 죽었을 것이다.
mask = ( rdf.pclass != 3 )
rdf['male_Third'] = mask.astype(int)

# 가정4: 50살 이상의 남자는 대다수가 죽었을 것이다.
mask = ( rdf.age < 50 ) 
rdf['male_age50'] = mask.astype(int)

# 분석에 활용할 열(속성)을 선택 
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked','women_child','male_fare0','male_Third','male_age50']]


# 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex', 'embarked'], axis=1, inplace=True)


# 속성(변수) 선택
X=ndf[['pclass', 'age', 'sibsp', 'parch',
       'women_child','male_fare0','male_Third','male_age50']]  #독립 변수 X
y=ndf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm
from sklearn.model_selection import GridSearchCV 
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf','poly', 'sigmoid','linear']}  
  
grid = GridSearchCV( svm.SVC(), param_grid, refit = True,cv=3, n_jobs = -1, verbose = 2 )

"""
estimator : classifier, regressor, pipeline이 사용될 수 있다.

param_grid : 파라미터 딕셔너리. (파라미터명과 사용될 여러 파라미터 값을 지정)

scoring : 예측 성능을 측정할 평가 방법. 보통은 사이킷런에서 제공하는 문자열 
        (예: ‘accuracy’)을 넣지만 별도의 함수도 직접 지정이 가능하다.

cv : 교차 검증을 위해 분할되는 폴드 수.

refit : True면 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 
        estimator 객체를 해당 하이퍼 파라미터로 재학습시킨다. (default:True)
"""

# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 248 tasks      | elapsed:   49.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.8min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [ ]:
# test data를 가지고 y_hat을 예측 (분류) 
y_hat = grid.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 1 0 0 1 1 0 0 0]
[0 0 1 0 0 1 1 1 0 0]


In [ ]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, y_hat)  
print(svm_matrix)
print('\n')

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, y_hat)            
print(svm_report)

[[108  17]
 [ 29  61]]


              precision    recall  f1-score   support

           0       0.79      0.86      0.82       125
           1       0.78      0.68      0.73        90

    accuracy                           0.79       215
   macro avg       0.79      0.77      0.78       215
weighted avg       0.79      0.79      0.78       215

